In [4]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image
import pandas as pd
import os
import numpy as np
import torch
torch.set_printoptions(precision=10)

from utils import get_image_embeddings, model_name_to_ids, load_models, model_dict

from app import data_path

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%load_ext autoreload
%autoreload 2


/Users/yonglinwu/dev/image-search-playground/venv/lib/python3.10/site-packages/gradio/outputs.py:43: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/Users/yonglinwu/dev/image-search-playground/venv/lib/python3.10/site-packages/gradio/inputs.py:217: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/Users/yonglinwu/dev/image-search-playground/venv/lib/python3.10/site-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/Users/yonglinwu/dev/image-search-playground/venv/lib/python3.10/site-packages/gradio/utils.py:875: UserWarning: Expected 1 arguments for function <function search at 0x2978ccee0>, received 2.
  warnings.warn(
/Users/yonglinwu/dev/image-search-playground/venv/lib/python3.10/s

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [2]:
patagonia_df = pd.read_csv(data_path + 'metadata/patagonia_losGatos.tsv', sep='\t')

In [3]:
patagonia_df.head()

,brand,title,product_url,price,description,size,category,colors,Poshmark,Unnamed: 9,...,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46,Unnamed: 47
0,Patagonia,Patagonia Women's Los Gatos Fleece 1/4-Zip Smo...,https://poshmark.com/listing/63d4821f2fbf1afe8...,$36.00,"A soft, warm and versatile quarter-zip pullove...",M,Tops,"[{'name': 'Gray', 'rgb': '#929292', 'message_i...",Poshmark,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Patagonia,Patagonia Los Gatos 1/4 Zip Pullover M Beech B...,https://poshmark.com/listing/63fcd7709f212bd48...,$59.00,"High pile, quarter zip pulllover\nMeasurements...",M,Tops,"[{'name': 'Brown', 'rgb': '#663509', 'message_...",Poshmark,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Patagonia,PATAGONIA Women's Los Gatos Fleece 1/4-Zip Pul...,https://poshmark.com/listing/642b9bbcfed51f812...,$59.00,PATAGONIA Women's Los Gatos Fleece 1/4-Zip Pul...,S,Tops,"[{'name': 'White', 'rgb': '#FFFFFF', 'message_...",Poshmark,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Patagonia,Girl’s Patagonia Los Gatos Fleece 1/4 Zip XS,https://poshmark.com/listing/63f4f459c5df6c7f8...,$30.00,Girl’s Patagonia Los Gatos 1/4 Zip Fleece\n\n-...,XSG,Other,"[{'name': 'Tan', 'rgb': '#d1b48e', 'message_id...",Poshmark,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Patagonia,Patagonia Los Gatos Quarter Zip Grey,https://poshmark.com/listing/622cc43d3a0db900b...,$59.00,Patagonia Los Gatos Quarter Zip Grey \nWomen’s...,M,Tops,"[{'name': 'Gray', 'rgb': '#929292', 'message_i...",Poshmark,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#download_images(patagonia_df, data_path)

In [56]:
load_models()

In [54]:
def generate_embeddings():
    embeddings_df = pd.DataFrame()

    # Get image embeddings
    with torch.no_grad():
        for fp in os.listdir(data_path + 'images/'):
            if fp.endswith('.jpg'):
                new_row = {'name': fp}
                for model_name in model_name_to_ids.keys():
                    new_row[f'{model_name}-embedding'] = get_image_embeddings(model_name, Image.open(data_path + 'images/' + fp))
                embeddings_df = embeddings_df.append(new_row, ignore_index=True)
    return embeddings_df

In [26]:
fp = os.listdir(data_path + 'images/')[0]

In [28]:
model_name = 'fashion'

In [29]:
new_row = {'name': fp, f'{model_name}-embedding': get_image_embeddings(model_name, Image.open(data_path + 'images/' + fp))}
                

In [57]:
embeddings_df = generate_embeddings()

In [58]:
embeddings_df

,name,sentence-transformer-clip-ViT-L-14-embedding,fashion-embedding,openai-clip-embedding
0,Women's Under Armour Hustle Fleece Hoodie pull...,"[1.0734258, 0.99022365, 0.32032806, 0.2895219,...","[0.23177437, -1.9268938, 0.273342, -0.02474568...","[-0.32902592, -0.09434131, 0.3055967, 0.229937..."
1,Patagonia Los Gatos Fleece Grey Pullover.jpg,"[0.6227796, 0.026531212, 0.45240527, -0.488214...","[0.38133767, -1.3040155, 1.1697398, -0.3085520...","[-0.1695469, 0.5067289, 0.31120676, -0.0083701..."
2,REI Women's Down With It Quilted Hooded Parka ...,"[0.8497103, 1.2925782, -0.21685322, 0.24116844...","[-0.30043703, -1.3144073, -0.33848628, 0.24008...","[-0.24841668, 0.4876942, 0.39810008, -0.141552..."
3,Chanel Haute Couture Navy Blue Dress Semi Shee...,"[0.536018, 0.60787296, -0.2751825, 1.0325747, ...","[-0.101031125, 0.033914, -0.44531134, -0.64656...","[-0.08328074, 0.19443086, 0.14361368, 0.259305..."
4,Patagonia Women’s S Los Gatos Quarter-Zip Flee...,"[0.79398394, 1.3899276, -0.21383175, 0.0109823...","[0.60070944, -1.1051046, 1.0572466, 0.47092092...","[-0.27894062, -0.09589732, 0.5556799, -0.13458..."
...,...,...,...,...
326,Women's REI Elements Jacket Size M.jpg,"[0.6310029, 0.9942212, 0.009293936, 0.7862729,...","[0.19858713, -1.8665266, -0.3323754, 0.0465058...","[-0.0952643, 0.8016211, 0.08129032, 0.15187423..."
327,CHANEL Black cotton bodycon tank dress with zi...,"[1.0761135, 0.18927886, -0.007131472, 0.625682...","[0.07516122, -0.1886161, 0.1334078, -0.2829321...","[-0.12297699, 0.026368856, 0.04415588, 0.26031..."
328,Reformation X Veda Women's Bad Leather Jacket ...,"[0.79690784, 1.2895226, 0.22802149, -0.2736021...","[-0.12224964, -0.38734418, 0.35824925, 0.95855...","[0.6507246, 0.27751687, 0.36114892, -0.0831387..."
329,DISNEY HER UNIVERSE LILO AND STICH Rainbow Qua...,"[1.1617887, 0.19193622, 0.046035454, 0.4334900...","[-0.20762922, 0.1754938, -0.7334341, -0.106492...","[-0.31946087, 0.19534132, 0.37351555, -0.09741..."


In [65]:
embeddings_path = os.path.join(data_path, 'metadata/patagonia_losGatos_embeddings.pq')
embeddings_df.to_parquet(embeddings_path)

In [66]:
embeddings_df = pd.read_parquet(embeddings_path)

In [67]:
for i, row in embeddings_df.iterrows():
    if '\n' in row['name']:
        print(row['name'])
        embeddings_df = embeddings_df.drop(i)

In [68]:
embeddings_df

,name,sentence-transformer-clip-ViT-L-14-embedding,fashion-embedding,openai-clip-embedding
0,Women's Under Armour Hustle Fleece Hoodie pull...,"[1.0734258, 0.99022365, 0.32032806, 0.2895219,...","[0.23177437, -1.9268938, 0.273342, -0.02474568...","[-0.32902592, -0.09434131, 0.3055967, 0.229937..."
1,Patagonia Los Gatos Fleece Grey Pullover.jpg,"[0.6227796, 0.026531212, 0.45240527, -0.488214...","[0.38133767, -1.3040155, 1.1697398, -0.3085520...","[-0.1695469, 0.5067289, 0.31120676, -0.0083701..."
2,REI Women's Down With It Quilted Hooded Parka ...,"[0.8497103, 1.2925782, -0.21685322, 0.24116844...","[-0.30043703, -1.3144073, -0.33848628, 0.24008...","[-0.24841668, 0.4876942, 0.39810008, -0.141552..."
3,Chanel Haute Couture Navy Blue Dress Semi Shee...,"[0.536018, 0.60787296, -0.2751825, 1.0325747, ...","[-0.101031125, 0.033914, -0.44531134, -0.64656...","[-0.08328074, 0.19443086, 0.14361368, 0.259305..."
4,Patagonia Women’s S Los Gatos Quarter-Zip Flee...,"[0.79398394, 1.3899276, -0.21383175, 0.0109823...","[0.60070944, -1.1051046, 1.0572466, 0.47092092...","[-0.27894062, -0.09589732, 0.5556799, -0.13458..."
...,...,...,...,...
326,Women's REI Elements Jacket Size M.jpg,"[0.6310029, 0.9942212, 0.009293936, 0.7862729,...","[0.19858713, -1.8665266, -0.3323754, 0.0465058...","[-0.0952643, 0.8016211, 0.08129032, 0.15187423..."
327,CHANEL Black cotton bodycon tank dress with zi...,"[1.0761135, 0.18927886, -0.007131472, 0.625682...","[0.07516122, -0.1886161, 0.1334078, -0.2829321...","[-0.12297699, 0.026368856, 0.04415588, 0.26031..."
328,Reformation X Veda Women's Bad Leather Jacket ...,"[0.79690784, 1.2895226, 0.22802149, -0.2736021...","[-0.12224964, -0.38734418, 0.35824925, 0.95855...","[0.6507246, 0.27751687, 0.36114892, -0.0831387..."
329,DISNEY HER UNIVERSE LILO AND STICH Rainbow Qua...,"[1.1617887, 0.19193622, 0.046035454, 0.4334900...","[-0.20762922, 0.1754938, -0.7334341, -0.106492...","[-0.31946087, 0.19534132, 0.37351555, -0.09741..."


In [8]:
import os

for fp in os.listdir(data_path + 'images/'):
    if '?' in fp:
        print(fp)

In [7]:
1+1

2